In [1]:
import os
# change directory in order to be able to import python scripts from the common package
if os.getcwd().endswith('crafter_rl'): os.chdir(os.path.expanduser(".."))

In [2]:
from env import VanillaEnv, CrafterReplayBuffer, create_local_semantic
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import crafter
from policy import ActorFCNet
import torch
import cv2

In [3]:
folder_name: str = './crafter_rl/experiments/231015-084433/'

def play_episode(model: ActorFCNet, seed=1)->bool:
    """
    returns true of the episode is concidered solved
    """
    env = crafter.Env( size=(512, 512))

    model.eval()
    done = False
    episode_return = 0
    obs = env.reset()
    frames = [obs]
    semantic = np.zeros((1, 9, 9), dtype=np.float32)
    while not done:       
        action_logits = model.forward(torch.FloatTensor(semantic).unsqueeze(0), contrastive=False)
        action = torch.argmax(action_logits)
        obs, rewards, done, info = env.step(action.item())
        semantic = create_local_semantic(
                info['semantic'], info['player_pos'][0], info['player_pos'][1],
                info['inventory']['health'], info['inventory']['food'],
                info['inventory']['drink'], info['inventory']['energy'],
                info['inventory']['sapling'], info['inventory']['wood'],
                info['inventory']['stone'], info['inventory']['coal'],
                info['inventory']['iron'], info['inventory']['diamond'],
                info['inventory']['wood_pickaxe'], info['inventory']['stone_pickaxe'],
                info['inventory']['iron_pickaxe'], info['inventory']['wood_sword'],
                info['inventory']['stone_sword'], info['inventory']['iron_sword'],
            )
        semantic = semantic.astype(np.float32).reshape((1, 9, 9))
        episode_return += rewards
        frames.append(obs)
    return frames, episode_return



models_names = [n for n in os.listdir(folder_name) if n.endswith('.pth')]
for models_name in models_names:
    model = ActorFCNet()

    ckp = torch.load(folder_name + models_name, map_location=torch.device('cpu'))
    model.load_state_dict(ckp['state_dict'])

    episode_return = 0
    while episode_return < 2:
        frames, episode_return = np.array(play_episode(model, VanillaEnv(seed=1, semantic=True)))
        print(episode_return)

    fps = 8
    out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (512,512))

    for i in range(len(frames)):
        rgb_img = cv2.cvtColor(frames[i], cv2.COLOR_RGB2BGR)
        out.write(rgb_img)
    out.release()

c:\Users\manus\miniconda3\envs\master\lib\site-packages\crafter\engine.py:126: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename.read_bytes())
c:\Users\manus\miniconda3\envs\master\lib\site-packages\crafter\worldgen.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tunnels = np.zeros(world.area, np.bool)
C:\Users\manus\AppData\Local\Temp\ipykernel_14248\2069232737.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested

-0.8999999999999999
0.10000000000000009
1.0999999999999999
-0.8999999999999999
2.1


C:\Users\manus\AppData\Local\Temp\ipykernel_18972\2610357289.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  frames, episode_return = np.array(play_episode(model, VanillaEnv(start_level=0, num_levels=0)))


-0.3921568627450911
0.5968627450980625
11.03843137254902
